In [1]:
import subprocess as sp
import sys,os,shutil,pickle,time
from os.path import join
from os.path import expanduser


In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('samsungbatterreview.csv')
df.head(5)

,stars,comment
0,5,\n\n\n\n\n\n\n\n\n\n \n \n \n The phone ...
1,5,Product delievery was late by one day but no i...
2,5,I was in a confusion to buy this or go for one...
3,1,Phone overheated some time also battery life w...
4,5,Awesome product by Samsung.Pros:Very Good audi...


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1200 entries, 0 to 1199
Data columns (total 2 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   stars    1200 non-null   int64 
 1   comment  1200 non-null   object
dtypes: int64(1), object(1)
memory usage: 18.9+ KB


In [5]:
df.describe().round(1)

,stars
count,1200.0
mean,4.2
std,1.4
min,1.0
25%,4.0
50%,5.0
75%,5.0
max,5.0


In [6]:
# Determine how many missing values exist in the collection, in which case you can use .sum() chained onto is.na()
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0


In [7]:
df= df.dropna()
df.shape

(1200, 2)

In [8]:
#pip install -q wordcloud
import wordcloud
from nltk.corpus import stopwords
import nltk
import string
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
stop = stopwords.words('english')

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/nalinjindal/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


In [9]:
df['stopwords'] = df['comment'].apply(lambda x: len([x for x in x.split() if x in stop]))
df[['comment','stopwords']].head()

,comment,stopwords
0,\n\n\n\n\n\n\n\n\n\n \n \n \n The phone ...,49
1,Product delievery was late by one day but no i...,151
2,I was in a confusion to buy this or go for one...,194
3,Phone overheated some time also battery life w...,22
4,Awesome product by Samsung.Pros:Very Good audi...,42


In [10]:
def count_punct(text):
    count = sum([1 for char in text if char in string.punctuation])
    return count

df['punctuation'] = df['comment'].apply(lambda x: count_punct(x))

In [11]:
df[['comment','punctuation']].head()

,comment,punctuation
0,\n\n\n\n\n\n\n\n\n\n \n \n \n The phone ...,49
1,Product delievery was late by one day but no i...,59
2,I was in a confusion to buy this or go for one...,75
3,Phone overheated some time also battery life w...,0
4,Awesome product by Samsung.Pros:Very Good audi...,12


In [12]:
df['hastags'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.startswith('#')]))
df[['comment','hastags']].head()

,comment,hastags
0,\n\n\n\n\n\n\n\n\n\n \n \n \n The phone ...,0
1,Product delievery was late by one day but no i...,0
2,I was in a confusion to buy this or go for one...,0
3,Phone overheated some time also battery life w...,0
4,Awesome product by Samsung.Pros:Very Good audi...,0


In [13]:
df.hastags.loc[df.hastags != 0].count()

0

In [14]:
df['numerics'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isdigit()]))
df[['comment','numerics']].head()

,comment,numerics
0,\n\n\n\n\n\n\n\n\n\n \n \n \n The phone ...,0
1,Product delievery was late by one day but no i...,8
2,I was in a confusion to buy this or go for one...,3
3,Phone overheated some time also battery life w...,3
4,Awesome product by Samsung.Pros:Very Good audi...,0


In [15]:
df['upper'] = df['comment'].apply(lambda x: len([x for x in x.split() if x.isupper()]))
df[['comment','upper']].head()

,comment,upper
0,\n\n\n\n\n\n\n\n\n\n \n \n \n The phone ...,2
1,Product delievery was late by one day but no i...,2
2,I was in a confusion to buy this or go for one...,29
3,Phone overheated some time also battery life w...,3
4,Awesome product by Samsung.Pros:Very Good audi...,0


In [16]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x.lower() for x in x.split()))
df['comment'].head()

0    the phone looks gorgeous....the build and desi...
1    product delievery was late by one day but no i...
2    i was in a confusion to buy this or go for one...
3    phone overheated some time also battery life w...
4    awesome product by samsung.pros:very good audi...
Name: comment, dtype: object

In [17]:
df['comment'] = df['comment'].str.replace('[^\w\s]','')
df['comment'].head()

<ipython-input-17-66a30b7fedcd>:1: FutureWarning: The default value of regex will change from True to False in a future version.
  df['comment'] = df['comment'].str.replace('[^\w\s]','')


0    the phone looks gorgeousthe build and design i...
1    product delievery was late by one day but no i...
2    i was in a confusion to buy this or go for one...
3    phone overheated some time also battery life w...
4    awesome product by samsungprosvery good audio ...
Name: comment, dtype: object

In [18]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop))
df['comment'].sample(10)

123     phone overheated time also battery life mark g...
542     confusion buy go oneplus 88pro confusion im sh...
824     awesome product samsungprosvery good audio qua...
982     confusion buy go oneplus 88pro confusion im sh...
600     phone looks gorgeousthe build design top notch...
447     overall good phone power packed performance fe...
1169    overall good phone power packed performance fe...
192     confusion buy go oneplus 88pro confusion im sh...
353     phone overheated time also battery life mark g...
585                phone excellent battery life upto mark
Name: comment, dtype: object

In [19]:
def remove_url(text):
    url = re.compile(r'https?://\S+|www\.\S+')
    return url.sub(r'', text)

In [20]:
# remove all urls from df
import re
import string

df['comment'] = df['comment'].apply(lambda x: remove_url(x))

In [21]:
def remove_html(text):
    html=re.compile(r'<.*?>')
    return html.sub(r'',text)

In [22]:
# remove all html tags from df
df['comment'] = df['comment'].apply(lambda x: remove_html(x))

In [23]:
# Reference : https://gist.github.com/slowkow/7a7f61f495e3dbb7e3d767f97bd7304b
def remove_emoji(text):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags 
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r'', text)

In [24]:
#Example
remove_emoji("Omg another Earthquake 😔😔")

'Omg another Earthquake '

In [25]:
# remove all emojis from df
df['comment'] = df['comment'].apply(lambda x: remove_emoji(x))

In [26]:
!pip install emot

Defaulting to user installation because normal site-packages is not writeable


In [27]:
from emot.emo_unicode import UNICODE_EMO, EMOTICONS

# Function for removing emoticons
def remove_emoticons(text):
    emoticon_pattern = re.compile(u'(' + u'|'.join(k for k in EMOTICONS) + u')')
    return emoticon_pattern.sub(r'', text)

In [28]:
#Example
remove_emoticons("Hello :-)")

'Hello '

In [29]:
df['comment'] = df['comment'].apply(lambda x: remove_emoticons(x))

In [30]:
from textblob import TextBlob
df['comment'][:5].apply(lambda x: str(TextBlob(x).correct()))

0    phone looks gorgeousthe build design top notch...
1    product delivery late one day issues thanks am...
2    confusion buy go onerous pro confusion in shar...
3    phone overheard time also battery life mark gi...
4    awesome product samsungprosvery good audit qua...
Name: comment, dtype: object

In [31]:
# We could do some of the cleaning steps as a sum of opreation like this:

# Apply a first round of text cleaning techniques
import re
import string

def clean_text_round1(text):
    '''Make text lowercase, remove text in square brackets, remove punctuation and remove words containing numbers.'''
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

round1 = lambda x: clean_text_round1(x)

In [32]:
df['comment'] = df.comment.apply(round1)
df.comment

0       phone looks gorgeousthe build design top notch...
1       product delievery late one day issues thanks a...
2       confusion buy go oneplus  confusion im sharing...
3       phone overheated time also battery life mark g...
4       awesome product samsungprosvery good audio qua...
                              ...                        
1195               phone excellent battery life upto mark
1196    overall good phone power packed performance fe...
1197    hhi guysi using flagship  march exact list poi...
1198     camera quality awesome premium lenses better ...
1199    solves purpose  terabyte internal storage must...
Name: comment, Length: 1200, dtype: object

In [33]:
# Apply a second round of cleaning
def clean_text_round2(text):
    '''Get rid of some additional punctuation and non-sensical text that was missed the first time around.'''
    text = re.sub('[‘’“”…]', '', text)
    text = re.sub('\n', '', text)
    return text

round2 = lambda x: clean_text_round2(x)

In [34]:
df['comment'] = df.comment.apply(round2)
df.comment

0       phone looks gorgeousthe build design top notch...
1       product delievery late one day issues thanks a...
2       confusion buy go oneplus  confusion im sharing...
3       phone overheated time also battery life mark g...
4       awesome product samsungprosvery good audio qua...
                              ...                        
1195               phone excellent battery life upto mark
1196    overall good phone power packed performance fe...
1197    hhi guysi using flagship  march exact list poi...
1198     camera quality awesome premium lenses better ...
1199    solves purpose  terabyte internal storage must...
Name: comment, Length: 1200, dtype: object

In [35]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:20]
freq

phone         3000
samsung       2640
one           2160
good          2160
battery       2040
screen        1560
camera        1560
life          1440
display       1320
ui            1320
use           1320
best          1200
better         960
time           960
quality        960
get            960
u              840
experience     840
also           840
features       840
dtype: int64

In [36]:
df['word_count'] = df['comment'].apply(lambda x: len(str(x).split(" ")))
df[['comment','word_count']].head()

,comment,word_count
0,phone looks gorgeousthe build design top notch...,96
1,product delievery late one day issues thanks a...,252
2,confusion buy go oneplus confusion im sharing...,249
3,phone overheated time also battery life mark g...,53
4,awesome product samsungprosvery good audio qua...,68


In [37]:
null_values=df.isna().sum()
null_values=pd.DataFrame(null_values,columns=['null'])
sum_tot=len(df)
null_values['percent']=null_values['null']/sum_tot*100
round(null_values,3).sort_values('percent',ascending=False)

,null,percent
stars,0,0.0
comment,0,0.0
stopwords,0,0.0
punctuation,0,0.0
hastags,0,0.0
numerics,0,0.0
upper,0,0.0
word_count,0,0.0


In [38]:
df['char_count'] = df['comment'].str.len() ## this also includes spaces
df[['comment','char_count']].head()

,comment,char_count
0,phone looks gorgeousthe build design top notch...,722
1,product delievery late one day issues thanks a...,1530
2,confusion buy go oneplus confusion im sharing...,1535
3,phone overheated time also battery life mark g...,319
4,awesome product samsungprosvery good audio qua...,557


In [39]:
def avg_word(sentence):
  words = sentence.split()
  return (sum(len(word) for word in words)/(len(words)+0.000001))

In [40]:
df['avg_word'] = df['comment'].apply(lambda x: avg_word(x)).round(1)
df[['comment','avg_word']].head()

,comment,avg_word
0,phone looks gorgeousthe build design top notch...,6.5
1,product delievery late one day issues thanks a...,5.4
2,confusion buy go oneplus confusion im sharing...,5.6
3,phone overheated time also battery life mark g...,5.3
4,awesome product samsungprosvery good audio qua...,7.4


In [41]:
df.sample(2)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
1113,1,phone overheated time also battery life mark g...,22,0,0,3,3,53,319,5.3
579,2,solves purpose terabyte internal storage must...,47,29,0,2,7,80,492,5.7


In [43]:
list(df)

['stars',
 'comment',
 'stopwords',
 'punctuation',
 'hastags',
 'numerics',
 'upper',
 'word_count',
 'char_count',
 'avg_word']

In [44]:
df.sample(5)

,stars,comment,stopwords,punctuation,hastags,numerics,upper,word_count,char_count,avg_word
525,5,phone excellent battery life upto mark,5,0,0,0,0,6,38,5.5
347,4,hhi guysi using flagship march exact list poi...,312,78,0,6,28,321,2082,5.9
1101,5,product delievery late one day issues thanks a...,151,59,0,8,2,252,1530,5.4
1145,5,confusion buy go oneplus confusion im sharing...,194,75,0,3,29,249,1535,5.6
343,1,phone overheated time also battery life mark g...,22,0,0,3,3,53,319,5.3


In [45]:
add_words = ["get",     
"use",     
"first",      
"like","u","goes","im"      
]

stop_words = set(stopwords.words("english"))
stop_added = stop_words.union(add_words)

In [46]:
df['comment'] = df['comment'].apply(lambda x: " ".join(x for x in x.split() if x not in stop_added))
df['comment'].sample(10)

770     phone looks gorgeousthe build design top notch...
846     overall good phone power packed performance fe...
226     overall good phone power packed performance fe...
154     awesome product samsungprosvery good audio qua...
184     awesome product samsungprosvery good audio qua...
534     awesome product samsungprosvery good audio qua...
68      camera quality awesome premium lenses better d...
1160    overall good phone power packed performance fe...
52      confusion buy go oneplus confusion sharing rev...
234     awesome product samsungprosvery good audio qua...
Name: comment, dtype: object

In [47]:
df1= df

In [48]:
mask = df1.comment.str.endswith('br') 
df1.loc[mask, 'comment'] = df1.loc[mask, 'comment'].str[:-2]

In [49]:
df1['comment'] = df1['comment'].str.rstrip('tty')

In [50]:
df1['comment'].apply(lambda x: x[:-2] if x.endswith('tty') else x)

0       phone looks gorgeousthe build design top notch...
1       product delievery late one day issues thanks a...
2       confusion buy go oneplus confusion sharing rev...
3       phone overheated time also battery life mark g...
4       awesome product samsungprosvery good audio qua...
                              ...                        
1195               phone excellent battery life upto mark
1196    overall good phone power packed performance fe...
1197    hhi guysi using flagship march exact list poin...
1198    camera quality awesome premium lenses better d...
1199    solves purpose terabyte internal storage must ...
Name: comment, Length: 1200, dtype: object

In [51]:
df1.loc[df1.comment.str.endswith('br'), 'comment']


Series([], Name: comment, dtype: object)

In [52]:
df1.loc[df1.punctuation >= 1000].comment.tolist()

[]

In [53]:
df.loc[df.punctuation >= 1000].comment.tolist()


[]

In [54]:
freq = pd.Series(' '.join(df['comment']).split()).value_counts()[:50]
freq

phone          3000
samsung        2640
one            2160
good           2160
battery        2040
screen         1560
camera         1560
life           1440
display        1320
ui             1320
best           1200
time            960
quality         960
better          960
features        840
experience      840
also            840
hours           720
rate            720
think           720
power           720
plus            720
performance     720
dont            720
used            600
scanner         600
fingerprint     600
mode            600
sensor          600
refresh         600
looks           600
feel            600
much            600
lot             480
phones          480
still           480
go              480
software        480
charging        480
want            480
exynos          480
point           480
day             480
options         480
updates         480
sound           480
less            480
user            480
great           480
version         480


In [55]:
df.to_csv('samsungbatteryreview2.csv', index=False)